In [1]:
import pandas as pd


df_initial=pd.read_csv("one_step.csv",index_col=False)
df_best=pd.read_csv("best_gpt_prompt_generations.csv",index_col=False)
for i in range(20):
    print(str(df_best["human 1 eval"][i])+"& "+str(df_best["human 2 eval"][i])+" & 0 &"+str(int(df_best["eval"][i]))+"\\\\")
    

8& 6 & 0 &8\\
6& 7 & 0 &8\\
4& 9 & 0 &8\\
6& 6 & 0 &5\\
5& 7 & 0 &8\\
9& 7 & 0 &8\\
4& 5 & 0 &7\\
6& 6 & 0 &7\\
6& 3 & 0 &7\\
6& 6 & 0 &7\\
6& 8 & 0 &8\\
6& 7 & 0 &7\\
4& 8 & 0 &8\\
5& 9 & 0 &8\\
5& 8 & 0 &7\\
6& 7 & 0 &7\\
4& 2 & 0 &2\\
6& 3 & 0 &7\\
6& 4 & 0 &8\\
4& 5 & 0 &7\\


In [2]:
lista=[6,	6,	2,	7,	5,	9,	4,	6,	5,	7,	6,	7,	5,	5,	6,	6,	4,	3,	7,	6,	5,	8,	6,	7,	6,	8,	8,	9,	7,	8,	7,	7,	6,	2,	2,	3,	6,	4,	6,	5]

df_initial["human 2 eval"]=[el for i, el in enumerate(lista) if i%2==0]
df_best["human 2 eval"]=[el for i, el in enumerate(lista) if i%2==1]

df_initial.to_csv("one_step.csv",index=False)
df_best.to_csv("best_gpt_prompt_generations.csv",index=False)

In [2]:
import numpy as np


print(np.mean(df_initial["eval"]))
print(np.mean(df_best["eval"]))
print(np.mean(df_initial["human 1 eval"]))
print(np.mean(df_best["human 1 eval"]))
print(np.mean(df_initial["human 2 eval"]))
print(np.mean(df_best["human 2 eval"]))

6.3
7.15
5.15
5.6
5.45
6.15


In [10]:
from sklearn.metrics import cohen_kappa_score
import numpy as np


annotator_scores = [
    df_initial["eval"],
    df_initial["human 1 eval"],
    df_initial["human 2 eval"]
]
kappa_matrix = np.zeros((3, 3))

for i in range(3):
    for j in range(i + 1, 3):
        kappa = cohen_kappa_score([annotator_scores[i][q] for q in range(20)],
                                  [annotator_scores[j][q] for q in range(20)])
        kappa_matrix[i][j] = kappa
        kappa_matrix[j][i] = kappa
print("Cohen's Kappa Matrix (initial instruction):")
print(kappa_matrix)
average_kappa = np.mean(kappa_matrix)
print(f"Average Cohen's Kappa (initial instruction): {average_kappa}")


annotator_scores = [
    df_best["eval"],
    df_best["human 1 eval"],
    df_best["human 2 eval"]
]
kappa_matrix = np.zeros((3, 3))

for i in range(3):
    for j in range(i + 1, 3):
        kappa = cohen_kappa_score([int(annotator_scores[i][q]) for q in range(20)],
                                  [int(annotator_scores[j][q]) for q in range(20)])
        kappa_matrix[i][j] = kappa
        kappa_matrix[j][i] = kappa
print("\n\nCohen's Kappa Matrix (GPT4 instruction):")
print(kappa_matrix)
average_kappa = np.mean(kappa_matrix)
print(f"Average Cohen's Kappa (GPT4 instruction): {average_kappa}")

Cohen's Kappa Matrix (initial instruction):
[[0.         0.07246377 0.28143713]
 [0.07246377 0.         0.0228013 ]
 [0.28143713 0.0228013  0.        ]]
Average Cohen's Kappa (initial instruction): 0.08371159928800914


Cohen's Kappa Matrix (GPT4 instruction):
[[0.         0.02061856 0.07692308]
 [0.02061856 0.         0.01162791]
 [0.07692308 0.01162791 0.        ]]
Average Cohen's Kappa (GPT4 instruction): 0.024259897911300406
